In [21]:
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import gensim
from gensim.models import word2vec
import string
import re
random.seed(1)
np.random.seed(1)

In [22]:
os.chdir('C:/Users/torna/Documents/StatApp/StatApp')
#os.chdir('C:/Users/Kim Antunez/Documents/Projets_autres')
#os.chdir('/Users/alainquartierlatente/Desktop/Ensae/StatApp')
#os.chdir('/home/aqlt/Documents/Ensae/StatApp')

In [23]:
df = pd.read_csv("data/sentimental_analysis/df_deft_sentiment.csv",sep='|',header=0,engine='python',encoding='UTF-8')

In [24]:
df = df[['tag','tweet']]

In [25]:
# On traite la mise en forme comme dans notre corpus
def mise_en_forme_phrase (phrase):
    phrase = phrase.lower()
    # On enlève la ponctuation + certaines apostrophes
    phrase = phrase.translate(str.maketrans('', '', string.punctuation + "'’«»—"))
    # On enlève les passages à la ligne
    phrase = re.sub('\\n', ' ', phrase)
    # On enlève les tabulations
    phrase = re.sub('\\t', ' ', phrase)
    # On enlève les espaces multiples et les espaces à la fin des phrases
    phrase = re.sub(' +', ' ', phrase)
    phrase = re.sub(' +$', '', phrase)
    phrase = re.sub('^ +', '', phrase)
    # phrase.isalpha() # inutile
    return(phrase)

In [54]:
#(df["tweet"] == "").value_counts()
df.iloc[2664]

tag        1
tweet    NaN
Name: 2664, dtype: object

In [68]:
df = df.loc[(pd.isnull(df["tweet"]))==False,:]

In [72]:
df['tag'].value_counts()

-1    13114
 0    12608
 1     7331
 2     2417
Name: tag, dtype: int64

In [74]:
corpus = []
for ligne in df.itertuples():
    if ligne.tag != 0:
        corpus.append([ligne.tag,mise_en_forme_phrase(ligne.tweet)])
corpus[0:2]

[[-1,
  'pff c est grave ce qui se passe à bruxelles on m a aussi volé les plans de mises à 4 voies des lignes rer de nivelles et ottignies'],
 [1,
  't heure mazri il était en mode daesh dans le métro les ptits babtous ils avaient raison de pas s approcher à côté de lui']]

In [77]:
len(corpus)

22862

In [95]:
corpus_modif = [[1 if score >= 1 else -1,tweet] for score, tweet in corpus]

In [96]:
val_pos = [[score,tweet] for score,tweet in corpus_modif if score == 1]
val_neg = [[score,tweet] for score,tweet in corpus_modif if score == -1]

In [98]:
len(val_neg)

13114

In [108]:
import random
from random import sample
random.seed(1)
corpus_train_pos = sample(val_pos,round(0.7*len(val_pos)))
corpus_test_pos = sample(val_pos,round(0.3*len(val_pos)))
corpus_train_neg = sample(val_neg,round(0.7*len(val_neg)))
corpus_test_neg = sample(val_neg,round(0.3*len(val_neg)))

In [109]:
corpus_train = corpus_train_pos + corpus_train_neg
corpus_test = corpus_test_pos + corpus_test_neg

# Calcul modèles sur Données SNCF

In [81]:
def calcul_sentence_embedding(phrase):
    if len(phrase)>0:
        sum1 = sum(matrice[mot] for mot in phrase if mot in model1.wv.index2word)
        #sum1 = sum(matrice_norm[mot] for mot in phrase if mot in model1.wv.index2word)
        sum2 = sum(matrice["lowfrequencyword"] for mot in phrase if mot not in model1.wv.index2word)
        #sum2 = sum(matrice_norm["lowfrequencyword"] for mot in phrase if mot not in model1.wv.index2word)
        return (sum1+sum2)/len(phrase)
    else:
        return matrice["lowfrequencyword"]

In [123]:
nom_dossier = 'ens'
# dim = 50
dim = 100
# dim = 300
epoch = 100
window = 4
learning_rate = 0.02
    
chemin1 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed1".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:]) 
model1 = gensim.models.keyedvectors.KeyedVectors.load(chemin1 + "/word2vec.model")
chemin2 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed5".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model2 = gensim.models.keyedvectors.KeyedVectors.load(chemin2 + "/word2vec.model")
chemin3 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed10".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model3 = gensim.models.keyedvectors.KeyedVectors.load(chemin3 + "/word2vec.model")
chemin4 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed15".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model4 = gensim.models.keyedvectors.KeyedVectors.load(chemin4 + "/word2vec.model")
chemin5 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed20".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:])  
model5 = gensim.models.keyedvectors.KeyedVectors.load(chemin5 + "/word2vec.model")
chemin6 = "data/{}/gensim/dim{}_ep{}_w{}_lr{}_seed25".format(nom_dossier, dim, epoch, window, str(learning_rate)[2:]) 
model6 = gensim.models.keyedvectors.KeyedVectors.load(chemin6 + "/word2vec.model")

matrice = {word : (model1.wv[word]+model2.wv[word]+model3.wv[word]+model4.wv[word]+model5.wv[word]+model6.wv[word])/6
           for word in model1.wv.index2word}

def normalize(v):
    norm = np.linalg.norm(v)
    if norm == 0: 
        return v
    return v / norm

matrice_norm = {word : normalize(matrice[word]) for word in matrice.keys()}

In [124]:
phrases = [[tweet.split(),score] for (score, tweet) in corpus_train]

In [125]:
phrases_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases]

In [126]:
Y_train = [score for (score, embedding) in phrases_emb]
X_train = [embedding for (score, embedding) in phrases_emb]

In [127]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
reg_log = lr.fit(X_train,Y_train)

In [128]:
Y_train_pred = reg_log.predict(X_train)

from sklearn import metrics
# Matrice confusion
cm = metrics.confusion_matrix(Y_train,Y_train_pred)
print(cm)
# Accuracy
acc = metrics.accuracy_score(Y_train,Y_train_pred)
print(acc) 

[[7359 1821]
 [2965 3859]]
0.7009497625593601


In [129]:
phrases_test = [[tweet.split(),score] for (score, tweet) in corpus_test]
phrases_test_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases_test]
Y_test = [score for (score, embedding) in phrases_test_emb]
X_test = [embedding for (score, embedding) in phrases_test_emb]
Y_test_pred = reg_log.predict(X_test)

In [130]:
# Matrice confusion
cm_test = metrics.confusion_matrix(Y_test,Y_test_pred)
print(cm_test)
# Accuracy
acc_test = metrics.accuracy_score(Y_test,Y_test_pred)
print(acc_test) 

[[3125  809]
 [1263 1661]]
0.6978710994459026


In [131]:
# Si on veut récup les fichiers
with open("data/sentimental_analysis/corpus_test.file", "rb") as f:
    corpus_test_git = pickle.load(f)
with open("data/sentimental_analysis/corpus_train.file", "rb") as f:
    corpus_train_git = pickle.load(f)

In [132]:
phrases = [[tweet.split(),score] for (score, tweet) in corpus_train_git]
phrases_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases]
Y_train_git = [score for (score, embedding) in phrases_emb]
X_train_git = [embedding for (score, embedding) in phrases_emb]

phrases_test = [[tweet.split(),score] for (score, tweet) in corpus_test_git]
phrases_test_emb = [[score,calcul_sentence_embedding(phrase)] for (phrase,score) in phrases_test]
Y_test_git = [score for (score, embedding) in phrases_test_emb]
X_test_git = [embedding for (score, embedding) in phrases_test_emb]

In [133]:
Y_test_git_pred = reg_log.predict(X_test_git)
# Matrice confusion
cm_test = metrics.confusion_matrix(Y_test_git,Y_test_git_pred)
print(cm_test)
# Accuracy
acc_test = metrics.accuracy_score(Y_test_git,Y_test_git_pred)
print(acc_test) 

[[ 951 1033]
 [ 485 1513]]
0.6187845303867403


In [134]:
reg_log_git = lr.fit(X_train_git,Y_train_git)
Y_train_git_pred = reg_log_git.predict(X_train_git)

from sklearn import metrics
# Matrice confusion
cm = metrics.confusion_matrix(Y_train_git,Y_train_git_pred)
print(cm)
# Accuracy
acc = metrics.accuracy_score(Y_train_git,Y_train_git_pred)
print(acc) 

[[17577  7423]
 [ 7983 17017]]
0.69188


In [135]:
Y_test_git_pred = reg_log_git.predict(X_test_git)
# Matrice confusion
cm_test = metrics.confusion_matrix(Y_test_git,Y_test_git_pred)
print(cm_test)
# Accuracy
acc_test = metrics.accuracy_score(Y_test_git,Y_test_git_pred)
print(acc_test) 

[[1399  585]
 [ 661 1337]]
0.6870919136112507


In [136]:
Y_test_pred = reg_log_git.predict(X_test)
# Matrice confusion
cm_test = metrics.confusion_matrix(Y_test,Y_test_pred)
print(cm_test)
# Accuracy
acc_test = metrics.accuracy_score(Y_test,Y_test_pred)
print(acc_test) 

[[3357  577]
 [1946  978]]
0.6321084864391951


In [138]:
with open("data/sentimental_analysis/reg_log_sncf.file", "wb") as f:
    pickle.dump(reg_log, f, pickle.HIGHEST_PROTOCOL)
with open("data/sentimental_analysis/reg_log_git.file", "wb") as f:
    pickle.dump(reg_log_git, f, pickle.HIGHEST_PROTOCOL)